# Отчёт
## Лабораторная работа №1
## Тема "Методы сбора открытых данных из сети интернет"
#### Герасимчук Михаил, p4141

# Parser


In [11]:
import requests
from lxml.html import fromstring
import json
import lxml.html

Получаем веб-страницу

In [19]:
def get_html(url):
    responce = requests.get(url)
    return responce.content

In [20]:
def get_tree(html_text):
    tree = fromstring(html_text, 'lxml')
    return tree

Получаем страницы


In [21]:
def get_table_pages(page_source):
    pages_links = page_source.xpath("//*[@class='item-link']/@href")
    return list(set(pages_links))

Получаем нужные нам данные 

In [22]:
def get_full_info(tree):
    skill = tree.xpath("//*[@class='skills-item']/text()")
    name =  tree.xpath("//*[@class='person-name']/p/text()")
    age = tree.xpath("//*[@class='age']/text()")
    gender = tree.xpath("//*[@class='gender']/text()")
    per_comp = tree.xpath("//*[@class='previous-company']/text()")
    salary = tree.xpath("//*[@class='salary']/text()")
    reg_date = tree.xpath("//*[@class='registered']/text()")
    data = [name, skill, age, gender, per_comp, salary, reg_date]
    return data

Структурируем данные в словарь

In [23]:
def get_item(table_data):
    result = dict()
    result = {'Name':table_data[0], 'Age': table_data[2], 'Gender':table_data[3],'Skills':table_data[1], 
              'Previous company':table_data[4], 'Salary':table_data[5], 'Register date':table_data[6]}
    return result

Сохраняем полученный результат

In [24]:
def save_to_json(file_path, npp_data):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(npp_data, f, ensure_ascii=False, indent=4)

Парсим

In [25]:
table_data = [[]]
data = dict()
for i in range (1, 348):
    html_text = get_html(f'https://iirmephi.ru/tutorials/jobhunt/{str(i)}')
    tree = get_tree(html_text)
    table_data.append(get_full_info(tree))
    merge_data = get_item(table_data[i])
    data[f'{i}'] = merge_data
save_to_json('npp_merge_data', data) 
    

In [19]:
data

{'1': {'Name': ['Mccormick Roy'],
  'Age': ['42'],
  'Gender': ['male'],
  'Skills': ['Hospitals',
   'Company Research',
   'Grant Writing',
   'Email',
   'Financial Risk',
   'Coaching',
   'Leadership Development',
   'Communication',
   'Job Description Development',
   'Project Planning'],
  'Previous company': ['Sendit'],
  'Salary': ['23000'],
  'Register date': ['2020-04-25']},
 '2': {'Name': ['Alexander Hayes'],
  'Age': ['47'],
  'Gender': ['male'],
  'Skills': ['Baseball',
   'Heavy Equipment',
   'Underwriting',
   'Oracle Database',
   'University Teaching',
   'Information Technology',
   'Team Building',
   'Outsourcing',
   'Golf',
   'Underwriting',
   'Integrated Marketing',
   'Linux',
   'Data Analysis'],
  'Previous company': ['Interse'],
  'Salary': ['28000'],
  'Register date': ['2020-06-15']},
 '3': {'Name': ['Stafford Berry'],
  'Age': ['27'],
  'Gender': ['male'],
  'Skills': ['Schedules',
   'Invoicing',
   'User Interface Design',
   'Kanban',
   'Windows',

#### В данном подходе мы работаем с веб страницами и для получения нужной информации используем xpath. После получения данных их можно обработать и сохранить в json формат. 

# VkAPI

In [46]:
import vk
import requests
import json
import time
import os
import vk_api
import csv

In [47]:
VK_ID = '51432483'
VK_TOKEN = 'bcfa95c2bcfa95c2bcfa95c24abfea59e1bbcfabcfa95c2dfd6d2c732d467b71a3c430d'

In [49]:
possible_targets = [53636214, 44625516, 180387873, 284284379]

Получаем информацию о нужном нам пользователе используя его id

In [50]:
def getFullInfoUser(id, VK_TOKEN):
    api = requests.get("https://api.vk.com/method/users.get", params = {
        "user_id": id,
        "access_token": VK_TOKEN,
        "v": 5.126,
        "fields": "domain, sex, bdate, city, country, home_town, education, followers_count, counters, photo_200_orig,relation, personal",
        "personal":"religion",
        "response": [
        ]
    })
    return json.loads(api.text)

api = getFullInfoUser(possible_targets[1], VK_TOKEN)

In [51]:
api

{'response': [{'id': 44625516,
   'domain': 'galustyanrf',
   'bdate': '25.10.1979',
   'city': {'id': 1, 'title': 'Moscow'},
   'country': {'id': 1, 'title': 'Russia'},
   'photo_200_orig': 'https://sun1-26.userapi.com/s/v1/ig2/Ri_MQaZAtdhw07ZP1XvIGp5Izs6A0QEe9F3760UOjtNB-fViBxfdyMrbkQfnkWllYxuPXi2paooPp-KCoKRPA8RM.jpg?size=200x200&quality=95&crop=212,53,415,415&ava=1',
   'followers_count': 1436982,
   'university': 815,
   'university_name': 'СГУ (бывш. СГУТиКД)',
   'faculty': 0,
   'faculty_name': '',
   'graduation': 2004,
   'education_form': 'Full-time',
   'education_status': 'Alumnus (Specialist)',
   'home_town': 'Сочи',
   'relation': 4,
   'relation_partner': {'id': 144658849,
    'first_name': 'Viktoria',
    'last_name': 'Galustyan'},
   'personal': {'alcohol': 0,
    'inspired_by': 'Звезды нашей вселенной',
    'life_main': 1,
    'people_main': 2,
    'political': 3,
    'smoking': 1},
   'counters': {'albums': 8,
    'audios': 282,
    'followers': 1436982,
    'gifts

Получаем id групп, на которые подписан пользователь

In [53]:
def getSubscriptionsGroups(VK_ID, VK_TOKEN):
    api = requests.get("https://api.vk.com/method/users.getSubscriptions", params = {
        "user_id": VK_ID,
        "count": 200,
        "access_token": VK_TOKEN,
        "v": 5.126,
        "response": {
            "groups": {
                "items": [{
                    "id": 1
                }]
            }
        }
    })
    return json.loads(api.text)["response"]["groups"]["items"]

groups = getSubscriptionsGroups(possible_targets[1], VK_TOKEN)
groups

[211006261,
 133441491,
 159839157,
 40553536,
 37322599,
 48512305,
 28490858,
 201822159,
 196774397,
 26662151,
 192394447,
 185661174,
 156601221]

#### В случае работы с VK API, мы делаем запрос к серверу, в котором указываем нужные нам параметры. На выходе получаем json формат, который в дальнейшей можем обрабатывать .
